In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_class 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
  AS
WITH class_code AS (
        SELECT * FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_avcdmdmm142
        WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_avcdmdmm142)
    ),
    class_name AS (
        SELECT * FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_avcdmdmm152
        WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_avcdmdmm152)
    )
    SELECT * EXCEPT (rn)
    FROM (SELECT ROW_NUMBER() OVER (PARTITION BY a1.objek, a1.class ORDER BY a1.proc_date DESC) rn,
                 a1.objek AS assortment,
                 a1.class AS assortment_class,
                 a2.kschg AS assortment_class_desc
          FROM class_code a1
          JOIN class_name a2 USING (clint)
          WHERE a2.spras = 'E')
    WHERE rn = 1
""")